In [1]:
suppressPackageStartupMessages(library(plyr))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
library(parallel)
library(ggpubr)
library(rstatix)
library(parallel)
library(ggpubr)
library(anndata)
library(ggsci)
library(ggrepel)
library(patchwork)


Attaching package: ‘ggpubr’


The following object is masked from ‘package:plyr’:

    mutate



Attaching package: ‘rstatix’


The following objects are masked from ‘package:plyr’:

    desc, mutate


The following object is masked from ‘package:stats’:

    filter




# get Aging related DEGs per cell type

In [15]:
file_list<-list.files(path ='/home//jupyter/BRI_Analysis/scRNA/Analysis-Cross-Sectional/02B_DESeq2_BR1_VS_BR2_L3', pattern = ".csv", all.files = FALSE,
           full.names = TRUE, recursive = FALSE,
           ignore.case = FALSE, include.dirs = TRUE, no.. = FALSE)

In [16]:
DEG_list<-mclapply(file_list,read.csv,mc.cores=length(file_list))
DEG<-do.call(rbind,DEG_list)

In [17]:
DEG<- DEG%>% mutate(Expression = case_when(log2FoldChange >=0.1 & padj <=  0.05 ~ "Up-regulated-BR1",
                                                     log2FoldChange <= -0.1 & padj <= 0.05 ~ "Down-regulated-BR1",
                                                     TRUE ~ "Unchanged"))

In [18]:
DEG_filtered<-DEG %>% filter(!Expression=='Unchanged')

# get LM res from chronological aging

In [19]:
file_list<-list.files(path ='/home//jupyter/BRI_Analysis/scRNA/Analysis-Longitudinal_Baselines/04B_LMM_Ages/', pattern = ".csv", all.files = FALSE,
           full.names = TRUE, recursive = FALSE,
           ignore.case = FALSE, include.dirs = TRUE, no.. = FALSE)

In [20]:
LMM_list<-mclapply(file_list,read.csv,mc.cores=length(file_list))
LMM_res<-do.call(rbind,LMM_list) %>%
  filter(factor == 'Ages') %>%
  group_by(celltype) %>%
  mutate(padj = p.adjust(`Pr...t..`, method = 'BH')) 
LMM_res$p.normal<-LMM_res$`Pr...t..`

In [21]:
LMM_res$Estimate<-as.numeric(LMM_res$Estimate)

In [22]:
LMM_res_aging<-LMM_res

# get LMM from 2 years time 

In [23]:
file_list<-list.files(path ='/home//jupyter/BRI_Analysis/scRNA/Analysis-Longitudinal_Baselines/04C_LMM_Time', pattern = ".csv", all.files = FALSE,
           full.names = TRUE, recursive = FALSE,
           ignore.case = FALSE, include.dirs = TRUE, no.. = FALSE)

In [24]:
LMM_list<-mclapply(file_list,read.csv,mc.cores=length(file_list))
LMM_res<-do.call(rbind,LMM_list) %>%
  filter(factor == 'sample.daysSinceFirstVisit') %>%
  group_by(celltype) %>%
  mutate(padj = p.adjust(`Pr...t..`, method = 'BH')) 
LMM_res$p.normal<-LMM_res$`Pr...t..`


In [25]:
LMM_res$Estimate<-as.numeric(LMM_res$Estimate)

In [26]:
LMM_res_time<-LMM_res

# 